# Problem 1

In [ ]:
import pandas as pd
import numpy as np 

def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""

    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

# Problem 2

In [ ]:
# Get Season dictionary
season_dd = dict(zip([str(one)+'-'+str(one+1)[2:] for one in range(1947, 2018)],
                     [str(one)+'-'+str(one+1)[2:] for one in range(1947, 2018)]))
season_dd

In [ ]:
## get all teams
params = {'LeagueID':'00'}
teams = get_nba_data('commonTeamYears', params)

## get all players
params = {'LeagueID':'00', 'Season': '2016-17', 'IsOnlyCurrentSeason': '0'}
players = get_nba_data('commonallplayers', params)

teams.ABBREVIATION = teams.ABBREVIATION.astype('category')
teams.TEAM_ID      = teams.TEAM_ID.astype('category')
teams.MIN_YEAR     = teams.MIN_YEAR.astype('int')
teams.MAX_YEAR     = teams.MAX_YEAR.astype('int')

teams = teams[teams.MAX_YEAR == 2017]
teams['TEAM_AGE'] = teams.MAX_YEAR - teams.MIN_YEAR

teams_clean = teams.copy() ## make a copy for later
teams.head()

In [ ]:
team_names = players[['TEAM_ABBREVIATION', 'TEAM_CODE']].drop_duplicates()#.set_index('TEAM_ABBREVIATION')

teams = pd.merge(teams_clean, team_names, left_on='ABBREVIATION', right_on='TEAM_ABBREVIATION')
teams.head()

In [ ]:
teams.TEAM_CODE = teams.TEAM_CODE.str.capitalize() # returns values so needs to be reassigned
teams.sort_values('ABBREVIATION', inplace=True)    # modifies object

In [ ]:
team_dd_text = teams.TEAM_ABBREVIATION+', '+teams.TEAM_CODE
team_dd = dict(zip(teams.TEAM_ID, team_dd_text))
team_dd

In [ ]:
## get all players for all seasons for each team
players_by_team_by_season = dict()

for i in season_dd.keys():
    params = {'LeagueID':'00', 'Season': i, 'IsOnlyCurrentSeason': '0'}
    players = get_nba_data('commonallplayers', params)

    players = players[players.TEAM_ID.isin(teams.TEAM_ID)]

    plyr_by_team_dd = dict()

    for t, p in players.groupby('TEAM_ID'):
        plyr_by_team_dd[team_dd[t]] = dict(zip(p.DISPLAY_LAST_COMMA_FIRST, p.PERSON_ID))

    players_by_team_by_season[i] = plyr_by_team_dd

In [ ]:
players_by_team_by_season

In [ ]:
# create dictionary for the teams in each season
teams_by_season_dd = dict()

for one in range(1947, 2018):
    team_copy = teams.copy()
    team_copy = team_copy[team_copy.MIN_YEAR <= one]
    
    team_dd_text = team_copy.TEAM_ABBREVIATION+', '+team_copy.TEAM_CODE
    team_dd1 = dict(zip(team_dd_text, team_copy.TEAM_ID))
    teams_by_season_dd[str(one)+'-'+str(one+1)[2:]] = team_dd1

In [ ]:
teams_by_season_dd

In [ ]:
from ipywidgets import interact, FloatSlider, Dropdown, Button

selected = 'BOS, Celtics'
selected_season = '2000-01'

season_menu = Dropdown(options=season_dd, value = selected_season)

team_menu = Dropdown(options=teams_by_season_dd[season_dd[selected_season]],
                    value = teams_by_season_dd[season_dd[selected_season]][selected])
plyr_by_team_dd = players_by_team_by_season[season_dd[selected_season]]
plyr_menu = Dropdown(options = plyr_by_team_dd[selected])

fetch_button = Button(description='Get Data!', icon='check')

display(season_menu,team_menu, plyr_menu, fetch_button)
## get data action
def get_data(change):
    #print(season_menu.value,team_menu.value, plyr_menu.value,opposing_team.value)
    params = {'PlayerID': plyr_menu.value,
          'PlayerPosition':'',
          'Season': season_menu.value,
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID': '0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': team_menu.value,
          'VsConference':'',
          'VsDivision':''}
    shotdata = get_nba_data('shotchartdetail', params)
    print(shotdata.head())
    
fetch_button.on_click(get_data)

# Problem 3

In [ ]:
from ipywidgets import interact, FloatSlider, Dropdown, Button

selected = 'BOS, Celtics'
selected_season = '2000-01'

season_menu = Dropdown(options=season_dd, value = selected_season)

team_menu = Dropdown(options=teams_by_season_dd[season_dd[selected_season]],
                    value = teams_by_season_dd[season_dd[selected_season]][selected])
plyr_by_team_dd = players_by_team_by_season[season_dd[selected_season]]
plyr_menu = Dropdown(options = plyr_by_team_dd[selected])

fetch_button = Button(description='Get Data!', icon='check')

display(season_menu,team_menu, plyr_menu, fetch_button)

# Update teams lists
def update_season(change):
    team_menu.index = None
    team_menu.options = teams_by_season_dd[change['new']]
    team_menu.value = teams_by_season_dd[change['new']][selected]
    plyr_menu.index = None
    plyr_by_team_dd = players_by_team_by_season[change['new']]
    plyr_menu.options = plyr_by_team_dd[selected]

def update_team(change):
    plyr_menu.index = None
    update = change['new']
    if team_menu.value is not None:
        plyr_by_team_dd = players_by_team_by_season[season_menu.value]
        plyr_menu.options = plyr_by_team_dd[team_dd[update]]

season_menu.observe(update_season, names='value')
team_menu.observe(update_team, names='value')

## get data action
def get_data(change):
    #print(season_menu.value,team_menu.value, plyr_menu.value,opposing_team.value)
    params = {'PlayerID': plyr_menu.value,
          'PlayerPosition':'',
          'Season': season_menu.value,
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID': '0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': team_menu.value,
          'VsConference':'',
          'VsDivision':''}
    shotdata = get_nba_data('shotchartdetail', params)
    print(shotdata.head())
    
fetch_button.on_click(get_data)

# Problem 4

## First Transformation

this will plot the shots made from each of the distances and will also plot the shots that were made

In [ ]:
## get every player that has played in the NBA
all_players = pd.DataFrame()
for i in season_dd.keys():
    params = {'LeagueID':'00', 'Season': i, 'IsOnlyCurrentSeason': '0'}
    iter_players = get_nba_data('commonallplayers', params)

    iter_players = iter_players[iter_players.TEAM_ID.isin(teams.TEAM_ID)]
    all_players = all_players.append(iter_players)

In [ ]:
all_players = all_players[['PERSON_ID','DISPLAY_LAST_COMMA_FIRST']].drop_duplicates()
player_dd = dict(zip(all_players.PERSON_ID,all_players.DISPLAY_LAST_COMMA_FIRST))

In [ ]:
player_dd

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, FloatSlider, Dropdown, Button

selected = 'BOS, Celtics'
selected_season = '2000-01'

season_menu = Dropdown(options=season_dd, value = selected_season)

team_menu = Dropdown(options=teams_by_season_dd[season_dd[selected_season]],
                    value = teams_by_season_dd[season_dd[selected_season]][selected])
plyr_by_team_dd = players_by_team_by_season[season_dd[selected_season]]
plyr_menu = Dropdown(options = plyr_by_team_dd[selected])

fetch_button = Button(description='Get Data!', icon='check')

display(season_menu,team_menu, plyr_menu, fetch_button)

# Update teams lists
def update_season(change):
    team_menu.index = None
    team_menu.options = teams_by_season_dd[change['new']]
    team_menu.value = teams_by_season_dd[change['new']][selected]
    plyr_menu.index = None
    plyr_by_team_dd = players_by_team_by_season[change['new']]
    plyr_menu.options = plyr_by_team_dd[selected]

def update_team(change):
    plyr_menu.index = None
    update = change['new']
    if team_menu.value is not None:
        plyr_by_team_dd = players_by_team_by_season[season_menu.value]
        plyr_menu.options = plyr_by_team_dd[team_dd[update]]

season_menu.observe(update_season, names='value')
team_menu.observe(update_team, names='value')

## get data action
def get_data(change):
    #print(season_menu.value,team_menu.value, plyr_menu.value,opposing_team.value)
    params = {'PlayerID': plyr_menu.value,
          'PlayerPosition':'',
          'Season': season_menu.value,
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID': '0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': team_menu.value,
          'VsConference':'',
          'VsDivision':''}
    shotdata = get_nba_data('shotchartdetail', params)

    ## code is from http://savvastjortjoglou.com/nba-shot-sharts.html
    def draw_court(ax=None, color='black', lw=1, outer_lines=False):
    
        from matplotlib.patches import Circle, Rectangle, Arc
        from matplotlib.pyplot import gca
    
        # If an axes object isn't provided to plot onto, just get current one
        if ax is None:
            ax = gca()

        # Create the various parts of an NBA basketball court

        # Create the basketball hoop
        # Diameter of a hoop is 18" so it has a radius of 9", which is a value
        # 7.5 in our coordinate system
        hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

        # Create backboard
        backboard = Rectangle((-30, -7.5), 60, 0, linewidth=lw, color=color)

        # The paint
        # Create the outer box 0f the paint, width=16ft, height=19ft
        outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                              fill=False)
        # Create the inner box of the paint, widt=12ft, height=19ft
        inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                              fill=False)

        # Create free throw top arc
        top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                             linewidth=lw, color=color, fill=False)
        # Create free throw bottom arc
        bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                                linewidth=lw, color=color, linestyle='dashed')
        # Restricted Zone, it is an arc with 4ft radius from center of the hoop
        restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                         color=color)

        # Three point line
        # Create the side 3pt lines, they are 14ft long before they begin to arc
        corner_three_a = Rectangle((-219, -47.5), 0, 140, linewidth=lw,
                                   color=color)
        corner_three_b = Rectangle((219, -47.5), 0, 140, linewidth=lw, color=color)
        # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
        # I just played around with the theta values until they lined up with the 
        # threes
        three_arc = Arc((0, 0), 475, 475, theta1=22.5, theta2=157.5, linewidth=lw,
                        color=color)

        # Center Court
        center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                               linewidth=lw, color=color)
        center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                               linewidth=lw, color=color)

        # List of the court elements to be plotted onto the axes
        court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                          bottom_free_throw, restricted, corner_three_a,
                          corner_three_b, three_arc, center_outer_arc,
                          center_inner_arc]

        if outer_lines:
            # Draw the half court line, baseline and side out bound lines
            outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                    color=color, fill=False)
            court_elements.append(outer_lines)

        # Add the court elements onto the axes
        for element in court_elements:
            ax.add_patch(element)

        return ax

    # create our jointplot
    joint_shot_chart = sns.jointplot(x="LOC_X", y="LOC_Y", data=shotdata, stat_func=None,
                                     kind='scatter', space=0, alpha=0.5)

    joint_shot_chart.fig.set_size_inches(12,11)

    # A joint plot has 3 Axes, the first one called ax_joint 
    # is the one we want to draw our court onto and adjust some other settings
    ax = joint_shot_chart.ax_joint
    draw_court(ax)

    # Adjust the axis limits and orientation of the plot in order
    # to plot half court, with the hoop by the top of the plot
    ax.set_xlim(-250,250)
    ax.set_ylim(422.5, -47.5)

    # Get rid of axis labels and tick marks
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.tick_params(labelbottom=False, labelleft=False)

    # Add a title
    ax.set_title(player_dd[plyr_menu.value]+ ' '+  season_menu.value + ' FGA', 
                 y=1.2, fontsize=18)

    # Add Data Scource and Author
    ax.text(-250,445,'Data Source: stats.nba.com'
            '\nAuthor: Savvas Tjortjoglou (savvastjortjoglou.com)',
            fontsize=12)
    print(shotdata.groupby('SHOT_ZONE_RANGE')['SHOT_MADE_FLAG'].mean())
    plt.show()
    
    
fetch_button.on_click(get_data)

## Transformation 2

This will get the average number of shots made per game in the specified season by the specified player

In [ ]:
selected = 'BOS, Celtics'
selected_season = '2000-01'

season_menu = Dropdown(options=season_dd, value = selected_season)

team_menu = Dropdown(options=teams_by_season_dd[season_dd[selected_season]],
                    value = teams_by_season_dd[season_dd[selected_season]][selected])
plyr_by_team_dd = players_by_team_by_season[season_dd[selected_season]]
plyr_menu = Dropdown(options = plyr_by_team_dd[selected])

fetch_button = Button(description='Get Data!', icon='check')

display(season_menu,team_menu, plyr_menu, fetch_button)

# Update teams lists
def update_season(change):
    team_menu.index = None
    team_menu.options = teams_by_season_dd[change['new']]
    team_menu.value = teams_by_season_dd[change['new']][selected]
    plyr_menu.index = None
    plyr_by_team_dd = players_by_team_by_season[change['new']]
    plyr_menu.options = plyr_by_team_dd[selected]

def update_team(change):
    plyr_menu.index = None
    update = change['new']
    if team_menu.value is not None:
        plyr_by_team_dd = players_by_team_by_season[season_menu.value]
        plyr_menu.options = plyr_by_team_dd[team_dd[update]]

season_menu.observe(update_season, names='value')
team_menu.observe(update_team, names='value')

## get data action
def get_data(change):
    #print(season_menu.value,team_menu.value, plyr_menu.value,opposing_team.value)
    params = {'PlayerID': plyr_menu.value,
          'PlayerPosition':'',
          'Season': season_menu.value,
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID': '0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID': team_menu.value,
          'VsConference':'',
          'VsDivision':''}
    shotdata = get_nba_data('shotchartdetail', params)
    shotdata.GAME_ID.astype('category')
    by_game = shotdata.groupby('GAME_ID')['SHOT_MADE_FLAG'].mean()
    plt.plot(by_game)
    plt.title('Percent of Shots made per game')
    print(by_game)
    plt.show()
    
fetch_button.on_click(get_data)